
Going to predict the spreads for games starting on march 17th


Going to preprocess data so its better!


Here's how you preprocess this:

make an array of ALL X's including training, testing, and prediction. Just preprocess this.


This means using the make neural net code, and including the prediction data including games in there. 

In [1]:

#import sys
#import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
#stderr = sys.stderr
#sys.stderr = open(os.devnull, 'w')

import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
from sklearn import preprocessing, cross_validation
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#This goes first 
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [ ]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    game.reshape(1,- 1)
    game = scale.transform(game)

    return game

In [ ]:
#here's where you create an array of all the games
games = game_maker 

In [7]:
def make_network(FILENAME,games):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    from sklearn.neural_network import MLPClassifier
    """Given the csv input of all the box scores, arrange it such that the home and away teams are lined up, 
    unnecessary columns removed, and proper hot encoding is done. Other stuff too probably.
    
    Note that this data has already been doctored from its original form, taking out most unnecessary columns but
    those could be useful later on.
    
    
    Parameters
    ----------
    FILENAME : file
        The csv of the data.
        
    Returns
    -------
    
    X_train : array
        Numpy array of the training inputs.
    y_train : array
        Numpy array of the training outputs.
    
    X_test : array
        Numpy array of the testing inputs.
    y_test : array
        Numpy array of the testing outputs.
    Sike!
    
    model : object
        MLP which can predict the outcome of NBA games
    """
    #Read in file, remove attempted and # and only account for % since that's more predictive in nature. 
    data = read_csv(FILENAME)
    data['3P%'] = np.divide(data['3P'].values,data['3PA'].values)
    del data['3P'],data['3PA']
    data['FG%'] = np.divide(data['FG'].values,data['FGA'].values)
    del data['FG'],data['FGA']
    data['FT%'] = np.divide(data['FT'].values,data['FTA'].values)
    del data['FT'],data['FTA']
    data = get_dummies(data)
    del data['VENUE_Home'],data['VENUE_Road']
    #print(data)
    
    

    dat = []

    for i in range(len(data.values)):
        data.values[i] = np.reshape(data.values[i],newshape=[1,len(data.values[i])])
    for p in range(int(len(data.values)/2)):
        fullboxgame = np.concatenate((data.values[2*p],data.values[(2*p)+1]))
        dat.append(fullboxgame)
    

    """At this point in the data dat is an array arranged as 
        OR  DR  TOT   A  PF  ST  TO  BL  PTS       3P%       FG%       FT% x2  (for road then home )
    so next up is to hot pull road and home points 
    
    road points is column 8 and home points is column 20. 
    
    So grab those from dat and make y. 
    
    X is every other column, so merge everyone else 
    
    """
    dat = np.array(dat)   
    roadpts = dat[:,8]

    homepts = dat[:,21]
    endspreadS = roadpts-homepts
    openingspreadS = dat[:,9]
#    print("opening spread:" + str(dat[:,9]))
    y = []
    for j in range(len(endspreadS)):
        openspread = openingspreadS[j]
       # print("this is the spread of the road team " + str(openspread))
        endspread = endspreadS[j]
       # print("the road team won by  .. " + str(endspread))
       # if endspread>openspread:
        #    y.append(np.array([0,1,0]))  #OK, now make sure this is formateed properly!
        if openspread + endspread < 0:
            y.append(np.array([0,1,0]))
        elif openspread + endspread > 0:
            y.append(np.array([1,0,0]))
        else: 
            y.append(np.array([0,0,1]))



    y = np.array(y)
    X1 = np.concatenate((dat[:,0:8],dat[:,10:21]),axis=1)
    X = np.concatenate((X1,dat[:,23:26]),axis=1)    #need to go one further column to snag HFT% 
    #print(X[0])
    scaler = preprocessing.StandardScaler().fit(X)
   
    X = scaler.transform(X)
    #print(X[0])
    

    X_train,X_test,y_train,y_test = cross_validation.train_test_split(X,y,test_size=0.27)
    #print((X[0]))
    #print(np.shape(X[0]))

    model = MLPClassifier()
    model.shuffle = True
    model.batch_size = 25
    #model.n_layers_ = 1000000
    #model.n_outputs_= 1000000
    #These don't do anything, have to adjust the layers in some different way! Keras is useful for this.
    model.fit(X_train,y_train)
    print(model.score(X_test,y_test))
    return model,scaler
    

If spread of road team is + that means they are supposed to lose by that much
if the end spread is - that means the road team lost by that much


In [3]:
def make_prediction_data(filename):
    from pandas import read_csv,get_dummies
    import numpy as np
    from sklearn import cross_validation
    data = read_csv(filename)
    #convert to per game stats and sort columns. 
    data['ORB'] =  np.divide(data['ORB'].values,data['G'].values)
    data['DRB'] =  np.divide(data['DRB'].values,data['G'].values)
    data['TRB'] =  np.divide(data['TRB'].values,data['G'].values)
    data['AST'] =  np.divide(data['AST'].values,data['G'].values)
    data['STL'] =  np.divide(data['STL'].values,data['G'].values)
    data['BLK'] =  np.divide(data['BLK'].values,data['G'].values)
    data['TOV'] =  np.divide(data['TOV'].values,data['G'].values)
    data['PF'] =  np.divide(data['PF'].values,data['G'].values)
    teams  = data['Team']

    data = data[['ORB' , 'DRB'  ,'TRB' ,  'AST' , 'PF' , 'STL' , 'TOV' , 'BLK' ,'3P%','FG%' ,'FT%']]
    print("Here is every teams index value: ")
    print(teams)
    return teams,data

In [29]:
def game_maker(roadteam,hometeam):
    """After creating a properly formated table, this concats the desired teams so they can be predicted. 
        Based on get team index # based on output of predictor, and make it the input for stats ie GSW are stats[0].
        and so on!
    """
    game = np.concatenate((roadteam,hometeam))
    game.reshape(1,- 1)
    game = scale.transform(game)

    return game

In [30]:
teams, stats =make_prediction_data('teamstats_319.csv')

Here is every teams index value: 
0     Golden State Warriors*
1     Houston Rockets*      
2     Toronto Raptors*      
3     New Orleans Pelicans  
4     Minnesota Timberwolves
5     Oklahoma City Thunder 
6     Denver Nuggets        
7     Cleveland Cavaliers   
8     Los Angeles Clippers  
9     Washington Wizards    
10    Charlotte Hornets     
11    Los Angeles Lakers    
12    Brooklyn Nets         
13    Portland Trail Blazers
14    Phoenix Suns          
15    Indiana Pacers        
16    Philadelphia 76ers    
17    Orlando Magic         
18    Boston Celtics*       
19    New York Knicks       
20    Atlanta Hawks         
21    Milwaukee Bucks       
22    Utah Jazz             
23    Dallas Mavericks      
24    Miami Heat            
25    San Antonio Spurs     
26    Chicago Bulls         
27    Detroit Pistons       
28    Sacramento Kings      
29    Memphis Grizzlies     
30    League Average        
Name: Team, dtype: object


In [32]:
game1 = game_maker(stats.values[1],stats.values[18])
game2 = game_maker(stats.values[3],stats.values[15])
game3 = game_maker(stats.values[7],stats.values[17])
game4 =  game_maker(stats.values[11],stats.values[2])
game5 = game_maker(stats.values[24],stats.values[4])
game6 = game_maker(stats.values[20],stats.values[22])
game7 = game_maker(stats.values[26],stats.values[12])


ValueError: Expected 2D array, got 1D array instead:
array=[ 9.22857143 34.17142857 43.4        21.75714286 19.8         8.62857143
 13.9         4.72857143  0.365       0.463       0.788       9.25714286
 35.21428571 44.47142857 22.44285714 20.1         7.17142857 13.87142857
  4.65714286  0.374       0.45        0.766     ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [21]:
noahbets,scale = make_network('1517_boxscores.csv')

0.7602256699576869


In [ ]:
noahbets.predict(np.array([game1,game2,game3,game4,game5,game6,game7]))

ATS Record 3//17:
*Mavs* vs. Nets X

Pistons vs. *Blazers*

Hawks vs. *Bucks* X

Pacers vs. *Wizards* Y

Rockets vs. *Pelicans* Y

*Hornets* vs. Knicks X

Cavs vs. *Bulls* Y

*Nuggets* vs. Grizzlies X

Twolves vs. *Spurs* 

Kings vs. *Jazz*

*Dubs* vs. Suns ***







Winners 3/10:
10 Hornets Y
24 Heat Y
27 Spurs N
26 Mavs Y
12 Clippers Y

Winners 3/11:
4 Raptors Y
25 Bulls Y
0 GSW N
3 Pelicans N
5 Nuggets Y
26 Rockets Y
13 Nets N
7 Cavs N
18 Celtics N

Winners 3/12:
Blazers Y
Spurs N
Bucks Y
Thunder Y


Winners 3/13:

Philly N
Wash N
OKC Y
Toronto Y
Dallas Y
Bulls N
Pels Y
Spurs Y
UTAH Y
Cle Y
Lakers Y

In [ ]:
game1 = game_maker(stats.values[24],stats.values[14])
game2 = game_maker(stats.values[27],stats.values[18])
game3 = game_maker(stats.values[21],stats.values[22])
game4 =  game_maker(stats.values[16],stats.values[10])
game5 = game_maker(stats.values[2],stats.values[3])
game6 = game_maker(stats.values[11],stats.values[19])
game7 = game_maker(stats.values[7],stats.values[26])
game8 = game_maker(stats.values[6],stats.values[29])
game9 = game_maker(stats.values[5],stats.values[25])
game10 =game_maker(stats.values[28],stats.values[23])
game11 =game_maker(stats.values[0],stats.values[15])

20-9

Predicted winners on March 9th:

Indiana
Chicago 
toronto
nop
utah
knicks    *
lakers
orlando  *
warriors
clippers


In [ ]:
20/29

Arrangement of Data:
Away :      OR  DR  TOT   A  PF  ST  TO  BL  PTS   3P%       FG%       FT%
  Home: OR  DR  TOT   A  PF  ST  TO  BL  3P%  FG%  FT%| AW HW
      0    1   2   3   4   5  6    7   8    9   10  

Data:
1. Road Orbs
2. Road Drbs
3. Road Trbs
4. Road Asts
5. Road Pfs
6. Road Stls
7. Road Tos. 
8. Road Blks 
9. Road 3p%
10. Road Fg%
11. Road Ft% 
12. Road venue Home (0)  REMOVED
13. Road venue Away (1) REMOVED
14. Home Orbs
15. Home Drbs
16. Home Trbs
17. Home Asts
18. Home Pfs
19. Home Stls
20. Home Tos. 
21. Home Blks 
22. Home 3p%
23. Home Fg%
24. Home Ft% 
25. Home venue Home (0) REMOVED
26. Home venue Away (1) REMOVED

With Outputs 

27. Home team win (1)
28. Road team win (0)


So you enter the statistics of both teams, and it gives a classification of whether or not the home or away team wins.
Unclear how good this actually is at characterizing teams. Should use team stats from as recent a metric as possible to reflect current gameplay


Let's do a raw version where I pull the last 5 stats of wiz vs pels 

3/8 Games and if it was correct:

Suns @ OKC YES

Nets @ HOR NO

Bos @ Min NO

Phi @ Mia YES

Sas @ GSW Says spurs, 

This is a good start. What's left?

1. What about assists, rebounds, steals, etc.? Can a table do that too? 

2. Hot encode wins and losses. Need to move that result column and delete everyhting except W and L.

3. Normalize data?

4. Create a net

5. Figure out prediction data, go by season averages? L5 averages? 

Remove unneccesary stats. Flatten 2 adjascent rows because they're the same game!

In [27]:
arr = np.array([10,2])

In [26]:
arr.reshape(1,-1)

array([[10,  2]])

In [28]:
arr

array([10,  2])